In [1]:
import pandas
import datetime
import numpy
import bs4
import urllib
import os

In [2]:
### Below are some useful urls to download data (ephemeris) files
### These may not be directly used in the code but can be used to
### download data that will be used in the code. Basically, noting
### them down here so that I wouldn't loose them!
# Info link
infoLink = "http://rbspgway.jhuapl.edu/analysis_models"
# TEST RBSP download link
testUrl = "https://www.rbsp-ect.lanl.gov/data_pub/rbspb/MagEphem/def/2013/rbspb_def_MagEphem_TS04D_20131231_v1.0.0.txt"
# TS07D Link:
ts07DUrl = "http://rbspgway.jhuapl.edu/SGMagEphem"

In [3]:
# THE TS07D Ephemeris files are in a folder by date
# Loop through the files in the folder, read the files,
# process the data (remove unwanted cols) and output 
# the data into a new file. That can be used by IDL to
# calculate AACGM coords.
# set the base directory
baseDir = "/home/bharat/Documents/code/rbsp-locs/"
# Loop through the base directory and sub directories
# and get a list of ephemeris files that we need!
fileList = []
for root, dirs, files in os.walk(baseDir):
    for fName in files:
        # Get only those files which start with 
        # the words "ts07d". These contain rbsp info!
        if fName.startswith("ts07d"):
            fileList.append( root + "/" + fName )
print "num files found--->", len(fileList)

num files found---> 1687


In [6]:
# Now loop through the files and work on getting the data
# setup a o/p file to write just the required params!
opFile = baseDir + "/rbspOp.txt"
cnt = 0
# Before appending to the file! delete it if it 
# exists already. We dont want to append to old data!
if os.path.isfile( opFile ):
    os.remove( opFile )
with open(opFile, 'a') as opf:
    for fn in fileList:
        currData = pandas.read_csv(fn, delim_whitespace=True)
        # Also need to check if curr file belongs 
        # to rbspa or rbspb. This can then be used
        # to identify which spacecraft Fps belong to!
        if "rbspa" in fn:
            currData["sat"] = "rbspa"
        else:
            currData["sat"] = "rbspb"
        currData["dtObj"] = pandas.to_datetime(currData["Time"])
        # Add a few additional datetime related columns
        currData["drvd_date"] = [ x.strftime("%Y%m%d") for x in currData["dtObj"] ]
        currData["drvd_time"] = [ x.strftime("%H%M") for x in currData["dtObj"] ]
        currData["drvd_year"] = [ x.year for x in currData["dtObj"] ]
        currData["drvd_mon"] = [ x.month for x in currData["dtObj"] ]
        currData["drvd_day"] = [ x.day for x in currData["dtObj"] ]
        # now we'll need only a few columns, just choose them
        currData = currData[ ['drvd_date', 'drvd_time',\
                              'drvd_year', 'drvd_mon',\
                              'drvd_day', 'Xgeo', 'Ygeo',\
                              'Zgeo', 'GeodLat', 'GeodLon',\
                              'GeodHeight', 'sat', 'CD_MLAT',\
                              'CD_MLON', 'CD_MLT'] ]
        # Write the DF data to the op file    
        currData.to_csv(opf, header=False, index=False, sep=' ')

       CD_MLAT     CD_MLON    CD_MLT
0   -15.006778  144.208238  4.868379
1   -15.093074  143.899375  4.931511
2   -15.174177  143.576870  4.993853
3   -15.250382  143.242106  5.055499
4   -15.321976  142.896323  5.116534
5   -15.389224  142.540695  5.177037
6   -15.452376  142.176336  5.237083
7   -15.511664  141.804310  5.296745
8   -15.567309  141.425639  5.356091
9   -15.619519  141.041304  5.415188
10  -15.668490  140.652257  5.474100
11  -15.714413  140.259422  5.532888
12  -15.757466  139.863702  5.591614
13  -15.797823  139.465985  5.650336
14  -15.835651  139.067144  5.709114
15  -15.871113  138.668048  5.768004
16  -15.904365  138.269560  5.827065
17  -15.935561  137.872547  5.886353
18  -15.964852  137.477880  5.945927
19  -15.992384  137.086440  6.005844
20  -16.018303  136.699125  6.066163
21  -16.042754  136.316849  6.126944
22  -16.065877  135.940555  6.188249
23  -16.087815  135.571211  6.250141
24  -16.108709  135.209826  6.312685
25  -16.128699  134.857447  6.375951
2